In [9]:
import com.microsoft.spark.fabric
from com.microsoft.spark.fabric.Constants import Constants

StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 11, Finished, Available, Finished)

In [10]:
df = spark.read.synapsesql("sil-warehouse.dbo.accounts")
df.showdf = spark.read.synapsesql("sil-warehouse.dbo.accounts")
df.show

StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 12, Finished, Available, Finished)

<bound method DataFrame.show of DataFrame[customer_id: string, balance: string, account_type: string, account_id: string]>

In [11]:
df.show()

StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 13, Finished, Available, Finished)

+-----------+-------+------------+----------+
|customer_id|balance|account_type|account_id|
+-----------+-------+------------+----------+
|         45|1000.50|     Savings|         1|
|         12|2500.75|    Checking|         2|
|         78|1500.00|     Savings|         3|
|         34|3000.25|    Checking|         4|
|         56| 500.00|     Savings|         5|
|         23|1200.50|    Checking|         6|
|         89| 800.75|     Savings|         7|
|         67|2200.00|    Checking|         8|
|         14| 900.25|     Savings|         9|
|         92|1800.50|    Checking|        10|
|          3|1100.75|     Savings|        11|
|         81|2700.00|    Checking|        12|
|         29|1300.25|     Savings|        13|
|         64|3200.50|    Checking|        14|
|         47| 700.75|     Savings|        15|
|         18|1400.00|    Checking|        16|
|         99| 600.25|     Savings|        17|
|          5|1600.50|    Checking|        18|
|         76| 400.75|     Savings|

In [12]:
create_table_query = """
CREATE TABLE if not exists Account_SCD (
    account_id int,
    customer_id int,
    account_type STRING,
    balance FLOAT,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Account_SCD'
"""

# Execute
spark.sql(create_table_query)

StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 14, Finished, Available, Finished)

DataFrame[]

In [13]:
from pyspark.sql.functions import *
df_src1= df.withColumn("hash_key",crc32(concat(*df.columns)))
display(df_src1)


StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f204f8a7-6d9d-44df-bd87-9b158490bc86)

In [14]:
from delta.tables import DeltaTable
target_path = "Files/Gold_layer/Account_SCD"
delta_target = DeltaTable.forPath(spark, target_path)
delta_target.toDF().show()

StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 16, Finished, Available, Finished)

+----------+-----------+------------+-------+----------+----------+--------------------+----------+--------------------+
|account_id|customer_id|account_type|balance|  hash_key|created_by|        created_date|updated_by|        updated_date|
+----------+-----------+------------+-------+----------+----------+--------------------+----------+--------------------+
|         1|         45|     Savings| 1000.5|3440579287|databricks|2025-05-04 18:13:...|databricks|2025-05-04 18:13:...|
|         2|         12|    Checking|2500.75|4140777957|databricks|2025-05-04 18:13:...|databricks|2025-05-04 18:13:...|
|         3|         78|     Savings| 1500.0| 457690895|databricks|2025-05-04 18:13:...|databricks|2025-05-04 18:13:...|
|         4|         34|    Checking|3000.25|1374993155|databricks|2025-05-04 18:13:...|databricks|2025-05-04 18:13:...|
|         5|         56|     Savings|  500.0|1549486685|databricks|2025-05-04 18:13:...|databricks|2025-05-04 18:13:...|
|         6|         23|    Chec

In [15]:
df_src1=df_src1.alias("src").join(delta_target.toDF().alias("tgt"),((col("src.account_id")==col("tgt.account_id"))&(col("src.hash_key")==col("tgt.hash_key"))),"anti").select(col("src.*"))
df_src1.show()

StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 17, Finished, Available, Finished)

+-----------+-------+------------+----------+--------+
|customer_id|balance|account_type|account_id|hash_key|
+-----------+-------+------------+----------+--------+
+-----------+-------+------------+----------+--------+



In [16]:
delta_target.alias("tgt").merge(df_src1.alias("src"),"tgt.account_id = src.account_id")\
    .whenMatchedUpdate(set={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, ff9e0896-57bb-4de5-9e17-6fa31babf9ca, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6f9910ee-b960-42e8-892e-2118a42de58c)

In [16]:
#customers
df_customers = spark.read.synapsesql("sil-warehouse.dbo.customers")
df_customers.show()

StatementMeta(, 6f480c62-09fb-4f03-a01c-180e3a878aec, 18, Finished, Available, Finished)

+-----------+-------------+-----------+------+-----+---------+---------------+
| first_name|         city|customer_id|   zip|state|last_name|        address|
+-----------+-------------+-----------+------+-----+---------+---------------+
|       John|      Toronto|          1|M4B1B3|   ON|      Doe|     123 Elm St|
|       Jane|       Ottawa|          2|K1A0B1|   ON|    Smith|  456 Maple Ave|
|    Michael|     Montreal|          3|H1A1A1|   QC|  Johnson|     789 Oak Dr|
|      Emily|      Calgary|          4|T2A0A1|   AB|    Davis|    101 Pine Rd|
|      David|    Vancouver|          5|V5K0A1|   BC|   Wilson| 202 Birch Blvd|
|       Emma|      Halifax|          6|B3H0A1|   NS|    Clark|   505 Cedar St|
|      James|     Winnipeg|          7|R3C0A1|   MB| Martinez|  606 Spruce Ln|
|     Olivia|     Edmonton|          8|T5A0A1|   AB|   Garcia|     707 Fir St|
|    William|     Victoria|          9|V8W0A1|   BC|    Lopez| 808 Redwood Dr|
|        Ava|  Quebec City|         10|G1A0A1|   QC|

In [15]:
create_table_query = """
CREATE TABLE if not exists Customer_SCD1 (
    customer_id int,
    first_name string,
    last_name string,
    address string,
    city string,
    state string,
    zip string,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Customer_SCD1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 6f480c62-09fb-4f03-a01c-180e3a878aec, 17, Finished, Available, Finished)

DataFrame[]

In [17]:
from pyspark.sql.functions import *
df_customers_src1= df_customers.withColumn("hash_key",crc32(concat(*df_customers.columns)))
display(df_customers_src1)

StatementMeta(, 6f480c62-09fb-4f03-a01c-180e3a878aec, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7e30c662-8823-4a6a-bac9-ac3e203cf674)

In [18]:
from delta.tables import DeltaTable
target_path = "Files/Gold_layer/Customer_SCD1"
delta_target = DeltaTable.forPath(spark, target_path)
delta_target.toDF().show()

StatementMeta(, 6f480c62-09fb-4f03-a01c-180e3a878aec, 20, Finished, Available, Finished)

+-----------+----------+---------+-------+----+-----+---+--------+----------+------------+----------+------------+
|customer_id|first_name|last_name|address|city|state|zip|hash_key|created_by|created_date|updated_by|updated_date|
+-----------+----------+---------+-------+----+-----+---+--------+----------+------------+----------+------------+
+-----------+----------+---------+-------+----+-----+---+--------+----------+------------+----------+------------+



In [20]:
df_customers_src1=df_customers_src1.alias("src").join(delta_target.toDF().alias("tgt"), ((col("src.customer_id") == col("tgt.customer_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_customers_src1.show()
     

StatementMeta(, 6f480c62-09fb-4f03-a01c-180e3a878aec, 22, Finished, Available, Finished)

+-----------+-------------+-----------+------+-----+---------+---------------+----------+
| first_name|         city|customer_id|   zip|state|last_name|        address|  hash_key|
+-----------+-------------+-----------+------+-----+---------+---------------+----------+
|       John|      Toronto|          1|M4B1B3|   ON|      Doe|     123 Elm St|2613897948|
|       Jane|       Ottawa|          2|K1A0B1|   ON|    Smith|  456 Maple Ave|3134331510|
|    Michael|     Montreal|          3|H1A1A1|   QC|  Johnson|     789 Oak Dr|3058880866|
|      Emily|      Calgary|          4|T2A0A1|   AB|    Davis|    101 Pine Rd| 537366123|
|      David|    Vancouver|          5|V5K0A1|   BC|   Wilson| 202 Birch Blvd|3300625524|
|       Emma|      Halifax|          6|B3H0A1|   NS|    Clark|   505 Cedar St|4197211656|
|      James|     Winnipeg|          7|R3C0A1|   MB| Martinez|  606 Spruce Ln|1040976427|
|     Olivia|     Edmonton|          8|T5A0A1|   AB|   Garcia|     707 Fir St| 668445207|
|    Willi

In [22]:
delta_target.alias("tgt").merge(df_customers_src1.alias("src"),"tgt.customer_id = src.customer_id")\
    .whenMatchedUpdate(set={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
     

StatementMeta(, 6f480c62-09fb-4f03-a01c-180e3a878aec, 24, Finished, Available, Finished)

In [24]:
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, 6f480c62-09fb-4f03-a01c-180e3a878aec, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f128810d-7f0a-4cf7-8a0a-3c08d65c557c)

Loan_payments


In [ ]:
import com.microsoft.spark.fabric
from com.microsoft.spark.fabric.Constants import Constants

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 9, Finished, Available, Finished)

In [26]:
df_payments = spark.read.synapsesql("sil-warehouse.dbo.loan_payments")
df_payments.show()

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 19, Finished, Available, Finished)

+----------+------------+--------------+-------+
|payment_id|payment_date|payment_amount|loan_id|
+----------+------------+--------------+-------+
|         1|  2024-01-01|        100.00|     45|
|         2|  2024-01-02|        150.00|     23|
|         3|  2024-01-03|        200.00|     67|
|         4|  2024-01-04|        250.00|     89|
|         5|  2024-01-05|        300.00|     12|
|         6|  2024-01-06|        350.00|     34|
|         7|  2024-01-07|        400.00|     56|
|         8|  2024-01-08|        450.00|     78|
|         9|  2024-01-09|        500.00|     90|
|        10|  2024-01-10|        550.00|     11|
|        11|  2024-01-11|        600.00|     22|
|        12|  2024-01-12|        650.00|     33|
|        13|  2024-01-13|        700.00|     44|
|        14|  2024-01-14|        750.00|     55|
|        15|  2024-01-15|        800.00|     66|
|        16|  2024-01-16|        850.00|     77|
|        17|  2024-01-17|        900.00|     88|
|        18|  2024-0

In [ ]:
create_table_query = """
CREATE TABLE if not exists Loan_Payments_SCD (
  payment_id int,
  loan_id int,
  payment_date timestamp,
  payment_amount float,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Files/Gold_layer/Loan_Payments_SCD'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 42, Finished, Available, Finished)

DataFrame[]

In [ ]:
from pyspark.sql.functions import *
df_loan_payment_src1= df_payments.withColumn("hash_key",crc32(concat(*df_payments.columns)))
display(df_loan_payment_src1)

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 005917a3-6783-42f5-ab98-36360ada2ad1)

In [ ]:
from delta.tables import DeltaTable
target_path = "Files/Gold_layer/Loan_Payments_SCD"
delta_target = DeltaTable.forPath(spark, target_path)
delta_target.toDF().show()

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 44, Finished, Available, Finished)

+----------+-------+------------+--------------+--------+----------+------------+----------+------------+
|payment_id|loan_id|payment_date|payment_amount|hash_key|created_by|created_date|updated_by|updated_date|
+----------+-------+------------+--------------+--------+----------+------------+----------+------------+
+----------+-------+------------+--------------+--------+----------+------------+----------+------------+



In [ ]:
df_loan_payment_src1=df_loan_payment_src1.alias("src").join(delta_target.toDF().alias("tgt"), ((col("src.payment_id") == col("tgt.payment_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_loan_payment_src1.show()
     

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 47, Finished, Available, Finished)

+----------+------------+--------------+-------+----------+
|payment_id|payment_date|payment_amount|loan_id|  hash_key|
+----------+------------+--------------+-------+----------+
|         1|  2024-01-01|        100.00|     45| 184599829|
|         2|  2024-01-02|        150.00|     23|2094682733|
|         3|  2024-01-03|        200.00|     67|  66449199|
|         4|  2024-01-04|        250.00|     89| 111748739|
|         5|  2024-01-05|        300.00|     12| 515047264|
|         6|  2024-01-06|        350.00|     34| 218410268|
|         7|  2024-01-07|        400.00|     56| 758438163|
|         8|  2024-01-08|        450.00|     78| 916774958|
|         9|  2024-01-09|        500.00|     90|4177002160|
|        10|  2024-01-10|        550.00|     11|3266558395|
|        11|  2024-01-11|        600.00|     22|1824065437|
|        12|  2024-01-12|        650.00|     33|3393622913|
|        13|  2024-01-13|        700.00|     44|1164454855|
|        14|  2024-01-14|        750.00|

In [ ]:
delta_target.alias("tgt").merge(df_loan_payment_src1.alias("src"),"tgt.payment_id = src.payment_id")\
    .whenMatchedUpdate(set={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()

display(spark.read.format("delta").option("header","true").load(target_path))
     

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 48, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7f999e7c-4671-4537-8cd1-92d4806055f9)

Loan

In [ ]:
df_loans = spark.read.synapsesql("sil-warehouse.dbo.loans")
df_loans.show()

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 23, Finished, Available, Finished)

+-------------+-----------+-----------+---------+-------+
|interest_rate|customer_id|loan_amount|loan_term|loan_id|
+-------------+-----------+-----------+---------+-------+
|          5.5|         45|   10000.50|       36|      1|
|          4.5|         12|   20000.75|       48|      2|
|          6.0|         78|   15000.00|       60|      3|
|          3.5|         34|   30000.25|       24|      4|
|          5.0|         56|   25000.00|       36|      5|
|          4.0|         23|   17500.50|       48|      6|
|          6.5|         89|   22500.75|       60|      7|
|          3.0|         67|   27500.00|       24|      8|
|          5.5|         14|   32500.25|       36|      9|
|          4.5|         92|   37500.50|       48|     10|
|          6.0|          3|   10000.75|       60|     11|
|          3.5|         81|   20000.00|       24|     12|
|          5.0|         29|   15000.25|       36|     13|
|          4.0|         64|   30000.50|       48|     14|
|          6.5

In [ ]:
create_table_query = """
CREATE TABLE if not exists Loans_SCD (
  loan_id int,
  customer_id int,
  interest_rate float,
  loan_term int,
  loan_amount float,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Files/Gold_layer/Loans_SCD'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 25, Finished, Available, Finished)

DataFrame[]

In [ ]:
df_loans_src1=df_loans.withColumn("hash_key",crc32(concat(*df_loans.columns)))
display(df_loans_src1)

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fe8dab01-2384-4753-9422-7f7ce7ad21e9)

In [ ]:
from delta.tables import DeltaTable
target_path = "Files/Gold_layer/Loans_SCD"
delta_target = DeltaTable.forPath(spark, target_path)
delta_target.toDF().show()

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 27, Finished, Available, Finished)

+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+
|loan_id|customer_id|interest_rate|loan_term|loan_amount|hash_key|created_by|created_date|updated_by|updated_date|
+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+
+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+



In [ ]:
df_loan1=df_loans_src1.alias("src").join(delta_target.toDF().alias("tgt"), ((col("src.loan_id") == col("tgt.loan_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_loan1.show()

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 29, Finished, Available, Finished)

+-------------+-----------+-----------+---------+-------+----------+
|interest_rate|customer_id|loan_amount|loan_term|loan_id|  hash_key|
+-------------+-----------+-----------+---------+-------+----------+
|          5.5|         45|   10000.50|       36|      1|3193460913|
|          4.5|         12|   20000.75|       48|      2|2948480094|
|          6.0|         78|   15000.00|       60|      3|2684399483|
|          3.5|         34|   30000.25|       24|      4|2501055163|
|          5.0|         56|   25000.00|       36|      5| 548789125|
|          4.0|         23|   17500.50|       48|      6|1990733294|
|          6.5|         89|   22500.75|       60|      7| 726435904|
|          3.0|         67|   27500.00|       24|      8|1449948090|
|          5.5|         14|   32500.25|       36|      9|2784571246|
|          4.5|         92|   37500.50|       48|     10|2974608584|
|          6.0|          3|   10000.75|       60|     11|3617366859|
|          3.5|         81|   2000

In [ ]:
delta_target.alias("tgt").merge(df_loan1.alias("src"),"tgt.loan_id = src.loan_id")\
    .whenMatchedUpdate(set={"tgt.loan_id":"src.loan_id","tgt.customer_id":"src.customer_id","tgt.loan_amount":"src.loan_amount","tgt.interest_rate":"src.interest_rate","tgt.loan_term":"src.loan_term","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.loan_id":"src.loan_id","tgt.customer_id":"src.customer_id","tgt.loan_amount":"src.loan_amount","tgt.interest_rate":"src.interest_rate","tgt.loan_term":"src.loan_term","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
     

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 31, Finished, Available, Finished)

In [ ]:
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 32, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fbb5211b-46a9-4b1e-8ead-3d3077f5e29e)

Transaction

In [ ]:
df_transactions = spark.read.synapsesql("sil-warehouse.dbo.transactions")
df_transactions.show()

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 33, Finished, Available, Finished)

+------------------+--------------+----------+----------------+----------------+
|transaction_amount|transaction_id|account_id|transaction_date|transaction_type|
+------------------+--------------+----------+----------------+----------------+
|            100.50|             1|        45|      2024-01-01|         Deposit|
|            200.75|             2|        12|      2024-01-02|      Withdrawal|
|            150.00|             3|        78|      2024-01-03|         Deposit|
|            300.25|             4|        34|      2024-01-04|      Withdrawal|
|            250.00|             5|        56|      2024-01-05|         Deposit|
|            175.00|             6|        23|      2024-01-06|      Withdrawal|
|            225.50|             7|        89|      2024-01-07|         Deposit|
|            275.75|             8|        67|      2024-01-08|      Withdrawal|
|            325.00|             9|        14|      2024-01-09|         Deposit|
|            375.25|        

In [ ]:
create_table_query = """
CREATE TABLE if not exists Transaction_SCD (
  transaction_id int,
  account_id int,
  transaction_date timestamp,
  transaction_amount float,
  transaction_type string,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Files/Gold_layer/Transaction_SCD'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 36, Finished, Available, Finished)

DataFrame[]

In [ ]:

df_transaction_src1=df_transactions.withColumn("hash_key",crc32(concat(*df_transactions.columns)))
display(df_transaction_src1)

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8ba75573-05e1-45f3-b284-07f2fef2b3b0)

In [ ]:

from delta.tables import DeltaTable
target_path = "Files/Gold_layer/Transaction_SCD"  
deltaTable = DeltaTable.forPath(spark, target_path)
deltaTable.toDF().show()
     

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 38, Finished, Available, Finished)

+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type|hash_key|created_by|created_date|updated_by|updated_date|
+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+
+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+



In [ ]:
df_tranaction1=df_transaction_src1.alias("src").join(deltaTable.toDF().alias("tgt"), ((col("src.transaction_id") == col("tgt.transaction_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_tranaction1.show()

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 39, Finished, Available, Finished)

+------------------+--------------+----------+----------------+----------------+----------+
|transaction_amount|transaction_id|account_id|transaction_date|transaction_type|  hash_key|
+------------------+--------------+----------+----------------+----------------+----------+
|            100.50|             1|        45|      2024-01-01|         Deposit|1353218021|
|            200.75|             2|        12|      2024-01-02|      Withdrawal|2483508441|
|            150.00|             3|        78|      2024-01-03|         Deposit|2634113635|
|            300.25|             4|        34|      2024-01-04|      Withdrawal|1948525431|
|            250.00|             5|        56|      2024-01-05|         Deposit|1082331057|
|            175.00|             6|        23|      2024-01-06|      Withdrawal| 178210813|
|            225.50|             7|        89|      2024-01-07|         Deposit|1690745351|
|            275.75|             8|        67|      2024-01-08|      Withdrawal|

In [ ]:
deltaTable.alias("tgt").merge(df_tranaction1.alias("src"),"tgt.transaction_id = src.transaction_id")\
    .whenMatchedUpdate(set={"tgt.transaction_id":"src.transaction_id","tgt.account_id":"src.account_id","tgt.transaction_date":"src.transaction_date","tgt.transaction_amount":"src.transaction_amount","tgt.transaction_type":"src.transaction_type","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.transaction_id":"src.transaction_id","tgt.account_id":"src.account_id","tgt.transaction_date":"src.transaction_date","tgt.transaction_amount":"src.transaction_amount","tgt.transaction_type":"src.transaction_type","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
     

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 40, Finished, Available, Finished)

In [ ]:
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, 5a784afc-068f-4bea-af6f-58406220b77c, 41, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a651920d-a5fe-4987-af47-d9b417b3c896)